In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.preprocessing import MinMaxScaler
from src.broker import Broker, BrokerException, BrokerValidationException
from src.krak_dealer import KrakDealer
from src.forex_broker import ForexBroker
from util import parse_configs, parse_args
from pykrakenapi.pykrakenapi import KrakenAPI, KrakenAPIError
from alpaca_trade_api.rest import REST, APIError
from oandapy import API, OandaError
from importlib import import_module
from sklearn.cluster import KMeans
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import pandas as pd
import numpy as np
import configparser
import os
import sys
import krakenex


config = configparser.ConfigParser()
try:
    config.read(os.path.relpath("../config.ini"))
except FileExistsError as e:
    print("FileExistsError: {}".format(e))
    sys.exit(1)

sid = SentimentIntensityAnalyzer()

# kraken
api = krakenex.API(key=config["kraken"]["api_key"], secret=config["kraken"]["private_key"])
kraken = KrakenAPI(api, tier="Starter")

In [4]:
# pair = "XBTUSDC"
pair = "BATUSD"
ohlc, last = kraken.get_ohlc_data(pair=pair)
ohlc.iloc[-1]
dataframe = ohlc.drop(columns=["vwap","volume","count"])
dataframe

,time,open,high,low,close
dtime,,,,,
2020-03-06 12:19:00,1583497140,0.24390,0.24390,0.24390,0.24390
2020-03-06 12:18:00,1583497080,0.24390,0.24390,0.24390,0.24390
2020-03-06 12:17:00,1583497020,0.24390,0.24390,0.24390,0.24390
2020-03-06 12:16:00,1583496960,0.24390,0.24390,0.24390,0.24390
2020-03-06 12:15:00,1583496900,0.24390,0.24390,0.24390,0.24390
...,...,...,...,...,...
2020-03-06 00:24:00,1583454240,0.23233,0.23233,0.23233,0.23233
2020-03-06 00:23:00,1583454180,0.23233,0.23233,0.23233,0.23233
2020-03-06 00:22:00,1583454120,0.23233,0.23233,0.23233,0.23233


In [3]:
pairs = kraken.get_tradable_asset_pairs()
pairs

,altname,wsname,aclass_base,base,aclass_quote,quote,lot,pair_decimals,lot_decimals,lot_multiplier,leverage_buy,leverage_sell,fees,fees_maker,fee_volume_currency,margin_call,margin_stop
ADAETH,ADAETH,ADA/ETH,currency,ADA,currency,XETH,unit,7,8,1,[],[],"[[0, 0.26], [50000, 0.24], [100000, 0.22], [25...","[[0, 0.16], [50000, 0.14], [100000, 0.12], [25...",ZUSD,80,40
ADAEUR,ADAEUR,ADA/EUR,currency,ADA,currency,ZEUR,unit,6,8,1,[2],[2],"[[0, 0.26], [50000, 0.24], [100000, 0.22], [25...","[[0, 0.16], [50000, 0.14], [100000, 0.12], [25...",ZUSD,80,40
ADAUSD,ADAUSD,ADA/USD,currency,ADA,currency,ZUSD,unit,6,8,1,[],[],"[[0, 0.26], [50000, 0.24], [100000, 0.22], [25...","[[0, 0.16], [50000, 0.14], [100000, 0.12], [25...",ZUSD,80,40
ADAXBT,ADAXBT,ADA/XBT,currency,ADA,currency,XXBT,unit,8,8,1,[],[],"[[0, 0.26], [50000, 0.24], [100000, 0.22], [25...","[[0, 0.16], [50000, 0.14], [100000, 0.12], [25...",ZUSD,80,40
ALGOETH,ALGOETH,ALGO/ETH,currency,ALGO,currency,XETH,unit,7,8,1,[],[],"[[0, 0.26], [50000, 0.24], [100000, 0.22], [25...","[[0, 0.16], [50000, 0.14], [100000, 0.12], [25...",ZUSD,80,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XXRPZJPY,XRPJPY,XRP/JPY,currency,XXRP,currency,ZJPY,unit,3,8,1,[],[],"[[0, 0.26], [50000, 0.24], [100000, 0.22], [25...","[[0, 0.16], [50000, 0.14], [100000, 0.12], [25...",ZUSD,80,40
XXRPZUSD,XRPUSD,XRP/USD,currency,XXRP,currency,ZUSD,unit,5,8,1,"[2, 3, 4, 5]","[2, 3, 4, 5]","[[0, 0.26], [50000, 0.24], [100000, 0.22], [25...","[[0, 0.16], [50000, 0.14], [100000, 0.12], [25...",ZUSD,80,40
XZECXXBT,ZECXBT,ZEC/XBT,currency,XZEC,currency,XXBT,unit,5,8,1,[],[],"[[0, 0.26], [50000, 0.24], [100000, 0.22], [25...","[[0, 0.16], [50000, 0.14], [100000, 0.12], [25...",ZUSD,80,40
XZECZEUR,ZECEUR,ZEC/EUR,currency,XZEC,currency,ZEUR,unit,3,8,1,[],[],"[[0, 0.26], [50000, 0.24], [100000, 0.22], [25...","[[0, 0.16], [50000, 0.14], [100000, 0.12], [25...",ZUSD,80,40


In [9]:
ticker = kraken.get_ticker_information(pair)
ticker

,a,b,c,h,l,o,p,t,v
XBTUSDC,"[8871.66000, 1, 1.000]","[8859.01000, 1, 1.000]","[8849.74000, 0.00953000]","[8914.20000, 8951.07000]","[8830.26000, 8611.08000]",8914.20000,"[8846.58182, 8787.73657]","[10, 70]","[0.30894204, 4.04696868]"


In [ ]:
def get_sentiment(text):
    """Given a text block, return a sentiment score based.

    :param text:
    :return:
    """
    text_polarity   = sid.polarity_scores(text)
    print(text_polarity)

    if text_polarity["compound"] > 0.50:
        sentiment   = "positive"

    else:
        sentiment   = "negative"

    return sentiment


In [ ]:
account = kraken.get_trade_balance()

attempt: 000 | ['EGeneral:Internal error']
attempt: 001 | ['EGeneral:Internal error']
attempt: 002 | ['EGeneral:Internal error']
attempt: 003 | ['EGeneral:Internal error']
attempt: 004 | ['EGeneral:Internal error']
attempt: 005 | ['EGeneral:Internal error']
attempt: 006 | ['EGeneral:Internal error']
attempt: 007 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
ca

attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 

call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal 

attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 

attempt: 001 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 

...

call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter

attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal 

call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter

call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter

attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal error']
call rate limiter exceeded (counter=16, limit=15) 
 sleeping for 5 seconds
call rate limiter exceeded (counter=15, limit=15) 
 sleeping for 5 seconds
attempt: 000 | ['EGeneral:Internal 

In [3]:
# # Feature Scaling Normalization
# close_prices = pd.DataFrame(minute_bars["close"])
# sc = MinMaxScaler(feature_range = (0, 1))
# training_set_scaled = sc.fit_transform(close_prices)
# 

In [4]:
# # calculate the max drawdown
# max_drawdown = max_drawdown(minute_bars["close"])
# day_bars.iloc[-50:]
# 

In [5]:
# day_spy_bars.iloc[-50:]
# 

In [6]:
# # calculate alpha and beta
# alpha = alpha(minute_bars["close"], minute_spy_bars["close"])
# beta = beta(minute_bars["close"], minute_spy_bars["close"])
# 

In [7]:
# print(alpha, beta, max_drawdown)